In [1]:
import pandas as pd
import requests
import json
import os
from dotenv import load_dotenv
import urllib

In [2]:
%load_ext dotenv
%dotenv
load_dotenv("BEERMAPPING_KEY")

True

In [3]:
apiKey = os.environ.get("BEERMAPPING_KEY")

In [4]:
baseURL = "http://beermapping.com/webservice/"

https://beermapping.com/api/reference/

Basic URL format: http://beermapping.com/webservice/{SERVICE}/API_KEY/{QUERY}  
Append '&s=json' to return JSON instead of XML

In [5]:
# # Search by City,State
# service = "loccity"

In [7]:
# Search by State
service = "locstate"
query = str(input("Enter 2-letter US State acronym:"))

Enter 2-letter US State acronym: ne


In [9]:
requestURL = f"{baseURL}{service}/{apiKey}/{query}&s=json"

In [11]:
response = requests.request("GET", requestURL)
print(response.status_code)
if response.status_code != 200:
    raise Exception(response.status_code, response.text)
jsonResponse = response.json()

200


In [13]:
type(jsonResponse)

list

In [34]:
# jsonResponse

In [137]:
jsonResponse['numberOfPages']

1

In [14]:
len(jsonResponse)

105

In [15]:
jsonResponse[0]

{'id': 21938,
 'name': '5168 Brewing',
 'status': 'Brewery',
 'reviewlink': 'https://beermapping.com/location/21938',
 'proxylink': 'http://beermapping.com/maps/proxymaps.php?locid=21938&amp;d=5',
 'blogmap': 'http://beermapping.com/maps/blogproxy.php?locid=21938&amp;d=1&amp;type=norm',
 'street': '5730 Hidcote Drive',
 'city': 'Lincoln',
 'state': 'NE',
 'zip': '68516',
 'country': 'United States',
 'phone': '402-875-5588',
 'url': '',
 'overall': '0',
 'imagecount': '0'}

In [30]:
jsonResponse[-1]

{'id': 21370,
 'name': 'Zipline Brewing Company',
 'status': 'Brewery',
 'reviewlink': 'https://beermapping.com/location/21370',
 'proxylink': 'http://beermapping.com/maps/proxymaps.php?locid=21370&amp;d=5',
 'blogmap': 'http://beermapping.com/maps/blogproxy.php?locid=21370&amp;d=1&amp;type=norm',
 'street': '721 N 14th St',
 'city': 'Omaha',
 'state': 'NE',
 'zip': '68102',
 'country': 'United States',
 'phone': '',
 'url': 'http%3A%2F%2Fziplinebrewing.com%2F',
 'overall': '0',
 'imagecount': '0'}

In [32]:
jsonResponse[0].keys()

dict_keys(['id', 'name', 'status', 'reviewlink', 'proxylink', 'blogmap', 'street', 'city', 'state', 'zip', 'country', 'phone', 'url', 'overall', 'imagecount'])

In [36]:
def getStateBreweries(state):
    '''
    Function to retrieve data about breweries within a US State
    '''
    # Assign URL Service to search by State:
    service = "locstate"
    # Assign search term:
    query = state
    # Assemble URL:
    requestURL = f"{baseURL}{service}/{apiKey}/{query}&s=json"
        # Note '&s=json' appended to return JSON formatted data
    
    response = requests.request("GET", requestURL)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    jsonResponse = response.json()
    
    return jsonResponse

In [37]:
state = str(input("Enter 2-letter US State acronym:"))
breweryJSON = getStateBreweries(state)

Enter 2-letter US State acronym: NE


200


In [38]:
breweryJSON[0].keys()

dict_keys(['id', 'name', 'status', 'reviewlink', 'proxylink', 'blogmap', 'street', 'city', 'state', 'zip', 'country', 'phone', 'url', 'overall', 'imagecount'])

In [76]:
def buildBreweryDB(listBreweries):
    '''
    Take a list of Brewery data in JSON format.
    Output a DF with key information about those breweries.
    '''
    # Define an empty dictionary, with keys:
    breweryDict = {'BreweryID': [], 'BreweryName': [], 'Type': [], 'ReviewLink': [],
                   'ProxyLink': [], 'Map': [], 'StreetAddress': [], 'City': [],
                   'State': [], 'Zip': [], 'Country': [], 'PhoneNum': [],
                   'Website': [], 'Rating': [], 'ImageCount': []}
    
    for b in listBreweries:
        breweryDict['BreweryID'].append(b['id'])
        breweryDict['BreweryName'].append(b['name'])
        breweryDict['Type'].append(b['status'])
        breweryDict['ReviewLink'].append(b['reviewlink'])
        breweryDict['ProxyLink'].append(b['proxylink'])
        breweryDict['Map'].append(b['blogmap'])
        breweryDict['StreetAddress'].append(b['street'])
        breweryDict['City'].append(b['city'])
        breweryDict['State'].append(b['state'])
        breweryDict['Zip'].append(b['zip'])
        breweryDict['Country'].append(b['country'])
        breweryDict['PhoneNum'].append(b['phone'])
#         breweryDict['Website'].append(b['url'])
        breweryDict['Website'].append(urllib.parse.unquote(b['url'])) # Fix parsed URLs
        breweryDict['Rating'].append(b['overall'])
        breweryDict['ImageCount'].append(b['imagecount'])
        
    return pd.DataFrame(breweryDict)

In [73]:
urllib.parse.unquote(test)

'http://www.5168brewing.com/'

In [77]:
breweryDF = buildBreweryDB(breweryJSON)

In [78]:
breweryDF.head()

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
0,21938,5168 Brewing,Brewery,https://beermapping.com/location/21938,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5730 Hidcote Drive,Lincoln,NE,68516,United States,402-875-5588,,0,0
1,22335,5168 Taproom,Beer Bar,https://beermapping.com/location/22335,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,3201 Farnam Street,Omaha,NE,68131,United States,402-934-5168,http://www.5168brewing.com/,0,0
2,21215,Alamo Draft House,Beer Bar,https://beermapping.com/location/21215,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,12750 Westport Pkwy,La Vista,NE,68138,United States,402-505-9979,https://drafthouse.com/omaha,0,0
3,19644,Backswing Brewing,Brewery,https://beermapping.com/location/19644,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,(402) 515-4263,facebook.com/BackswingBrewingCo/timeline,0,0
4,21194,Backswing Brewing Company,Brewery,https://beermapping.com/location/21194,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,402-515-4263,,0,0


In [57]:
test = breweryDF.loc[1][12]
test

'http%3A%2F%2Fwww.5168brewing.com%2F'

In [58]:
from requests.utils import requote_uri

In [59]:
requote_uri(test)

'http%3A%2F%2Fwww.5168brewing.com%2F'

In [61]:
test.decode('UTF-8')

AttributeError: 'str' object has no attribute 'decode'

In [63]:
from urllib.parse import urlparse

In [69]:
from urllib.parse import urlunparse

In [65]:
import urllib

In [66]:
urllib.parse.quote(test)

'http%253A%252F%252Fwww.5168brewing.com%252F'

In [70]:
urllib.parse.urlunparse(test)

ValueError: too many values to unpack (expected 7)

In [72]:
urllib.unquote(test.encode("utf8"))

AttributeError: module 'urllib' has no attribute 'unquote'

In [73]:
urllib.parse.unquote(test)

'http://www.5168brewing.com/'

We have a winner!!

In [75]:
# Probably just this?
import urllib

In [45]:
# breweryDict = {'BreweryID': [], 'BreweryName': [], 'Type': [], 'ReviewLink': [],
#                    'ProxyLink': [], 'Map': [], 'StreetAddress': [], 'City': [],
#                    'State': [], 'Zip': [], 'Country': [], 'PhoneNum': [],
#                    'Website': [], 'Overall': [], 'ImageCount': []}

In [43]:
# breweryDict = {'id': [], 'name': [], 'status': [], 'reviewlink': [],
#                    'proxylink': [], 'blogmap': [], 'street': [], 'city': [],
#                    'state': [], 'zip': [], 'country': [], 'phone': [],
#                    'url': [], 'overall': [], 'imagecount': []}

In [46]:
# for b in breweryJSON:
#     breweryDict['BreweryID'].append(b['id'])
#     breweryDict['BreweryName'].append(b['name'])
#     breweryDict['Type'].append(b['status'])
#     breweryDict['ReviewLink'].append(b['reviewlink'])
#     breweryDict['ProxyLink'].append(b['proxylink'])
#     breweryDict['Map'].append(b['blogmap'])
#     breweryDict['StreetAddress'].append(b['street'])
#     breweryDict['City'].append(b['city'])
#     breweryDict['State'].append(b['state'])
#     breweryDict['Zip'].append(b['zip'])
#     breweryDict['Country'].append(b['country'])
#     breweryDict['PhoneNum'].append(b['phone'])
#     breweryDict['Website'].append(b['url'])
#     breweryDict['Rating'].append(b['overall'])
#     breweryDict['ImageCount'].append(b['imagecount'])

In [50]:
# breweryDict

In [48]:
# df = pd.DataFrame(breweryDict)

In [49]:
# df.head()

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Overall,ImageCount
0,21938,5168 Brewing,Brewery,https://beermapping.com/location/21938,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5730 Hidcote Drive,Lincoln,NE,68516,United States,402-875-5588,,0,0
1,22335,5168 Taproom,Beer Bar,https://beermapping.com/location/22335,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,3201 Farnam Street,Omaha,NE,68131,United States,402-934-5168,http%3A%2F%2Fwww.5168brewing.com%2F,0,0
2,21215,Alamo Draft House,Beer Bar,https://beermapping.com/location/21215,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,12750 Westport Pkwy,La Vista,NE,68138,United States,402-505-9979,https%3A%2F%2Fdrafthouse.com%2Fomaha,0,0
3,19644,Backswing Brewing,Brewery,https://beermapping.com/location/19644,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,(402) 515-4263,facebook.com%2FBackswingBrewingCo%2Ftimeline,0,0
4,21194,Backswing Brewing Company,Brewery,https://beermapping.com/location/21194,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,402-515-4263,,0,0


In [18]:
# breweriesStr = json.dumps(jsonResponse, indent=4, sort_keys=True)

In [25]:
# type(breweriesStr)

In [26]:
# breweriesStr[:500]

In [22]:
# breweriesJSON = json.loads(breweriesStr)

In [27]:
# type(breweriesJSON)

In [29]:
# breweriesJSON[0]

## DataFrame time:

In [79]:
breweryDF.head()

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
0,21938,5168 Brewing,Brewery,https://beermapping.com/location/21938,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5730 Hidcote Drive,Lincoln,NE,68516,United States,402-875-5588,,0,0
1,22335,5168 Taproom,Beer Bar,https://beermapping.com/location/22335,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,3201 Farnam Street,Omaha,NE,68131,United States,402-934-5168,http://www.5168brewing.com/,0,0
2,21215,Alamo Draft House,Beer Bar,https://beermapping.com/location/21215,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,12750 Westport Pkwy,La Vista,NE,68138,United States,402-505-9979,https://drafthouse.com/omaha,0,0
3,19644,Backswing Brewing,Brewery,https://beermapping.com/location/19644,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,(402) 515-4263,facebook.com/BackswingBrewingCo/timeline,0,0
4,21194,Backswing Brewing Company,Brewery,https://beermapping.com/location/21194,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,402-515-4263,,0,0


In [80]:
breweryDF.Type.unique()

array(['Brewery', 'Beer Bar', 'Beer Store', 'Brewpub', 'Homebrew'],
      dtype=object)

Save a backup, in case the site goes down again:

In [81]:
breweryDF.to_csv('beerMapping.csv', index=False)